In [1]:
import os
import errno
import re
import itertools
import pprint
import cx_Oracle

directory =  os.getcwd()
inputDirectory = os.path.join(directory, './in')
outputDirectory = os.path.join(directory, './out')
schemaVersionSql = 'SELECT MAX(version) FROM SPATIALNETFM_VERSION ORDER BY DATE_INSTALLED DESC'

def dictToFile(pathAndFilename, dictionary):
    with open(pathAndFilename, "w") as file:
        for k, v in dictionary.items():
            dictionary_content = k + ": " + v + "\n"
            file.write(dictionary_content)

# create subdirectory
try:
    os.makedirs(inputDirectory)
except OSError as exception:
    if exception.errno != errno.EEXIST:
        raise
        
try:
    os.makedirs(outputDirectory)
except OSError as exception:
    if exception.errno != errno.EEXIST:
        raise

count = 0
        
for filename in os.listdir(inputDirectory): # count list of ini files
    if ".ini" in filename or ".INI" in filename:
        count += 1
        
        print(os.path.join(inputDirectory, filename))
        
        with open(os.path.join(inputDirectory, filename), "r") as fileObj:
            fileData = fileObj.read()
            
        fileData = dict(re.findall(r'(.+)=(.+\S)', fileData, re.MULTILINE)) # capture everything before equals and after equals but only up until the last actual character, not whitespace
        
        #print pprint.pprint(fileData)
        
        newList = []
        try:
            newList.append(('database_source',fileData["DATABASE_NAME"].lstrip(' ')))
            newList.append(('database_user',fileData["USERNAME"].lstrip(' ')))
            newList.append(('database_password',fileData["PASSWORD"].lstrip(' ')))

            connectionString = '{}/{}@{}'.format(fileData["USERNAME"].lstrip(' '), fileData["PASSWORD"].lstrip(' '), fileData["DATABASE_NAME"].lstrip(' '))
            #print connectionString
            
            try:
                # Make connection to oracle
                conn = cx_Oracle.connect(connectionString)
                c = conn.cursor()

                # Execute the sql query
                c.execute(schemaVersionSql)

                # Get all output of the query as a list
                data = c.fetchall()  
            except exception as e:
                data = None
            
            # If data is returned
            if data:
                version = data[0][0]
                #print version
                if '6.2.2.9' in version: newList.append(('spatial_model_version', 'v6_2_2_9'))
                elif '7.1.0.0' in version: newList.append(('spatial_model_version', 'v7_1_0'))
                elif '8.1.0.0' in version: newList.append(('spatial_model_version', 'v8_1_0'))
                else: newList.append(('spatial_model_version', ''))
            else:
            # If the query doesn't return data - The index doesn't exist
                newList.append(('spatial_model_version', ''))

            print(pprint.pprint(newList))
            
            dictToFile(os.path.join(outputDirectory, os.path.splitext(filename)[0] + '.yml'), dict(newList))
            
        except exception as ex:
            print(ex)
        
    else:
        continue

if count is 0:
    print('No ini files found in "./in" directory')

No ini files found in "./in" directory
